<a href="https://colab.research.google.com/github/Muhamadsulton/kaggle/blob/main/%D0%93%D1%80%D1%8F%D0%B7%D0%BD%D1%8B%D0%B5_%D1%82%D0%B0%D1%80%D0%B5%D0%BB%D0%BA%D0%B8_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
import zipfile
with zipfile.ZipFile('../input/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')

In [ ]:
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

In [ ]:
data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

In [ ]:
import shutil 
from tqdm import tqdm

In [ ]:
train_dir = 'train'#создается папка для тренировки и каждая  фотка кладется в папку для волидации
val_dir = 'val'#папка для валидациии
class_names = ['cleaned', 'dirty']

In [ ]:
#вот эта ячейка берет каждую 6 фотку и кладет ее в папку для валидации
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [ ]:
!ls train

In [ ]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
from torchvision import transforms, models
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),#выразание участков изображения(аугументация) и отресайзит до размера 224
    transforms.RandomHorizontalFlip(),#отзеркалим относительно горизонтали
    transforms.ToTensor(),#превращаем в тензор все
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#нормализация картинов по этим константам есть на сайте пайторча
])
])

In [ ]:
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),#ужимаем изображение до размера 224 на 224
    transforms.ToTensor(),#превращаем изображение в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#нормализация картинов по этим константам есть на сайте пайторча
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)#ImageFolder передает изображение в виде тензора,если он с 1 агрументом то не в виде тензора
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader( #на вход принимает imageFolder перемещивает датасет
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)a
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)


In [ ]:
#проверяем делиться ли на цело количество изображений на батч, то последний батч будет не полным и он не участвует в обучении нейронки
len(train_dataloader), len(train_dataset)

Чтобы посмотреть на это высчитываем все обратно т.е. нормировка обраьная

In [ ]:
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean);#permute нужен потому что тензор меняет размерность а пермют возвражает его в начальный вид

Посмотрим изображения

In [ ]:
def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [ ]:
#функция для обучения нейросети имеет две фазы такие как валидация и тренировка
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):#иттератор по эпохам
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':#если фаза равна тренировки то она делает градиентные шаги
                dataloader = train_dataloader #дает нам батчи
                scheduler.step() #делает градиентный шаг с который уменьшается через каждые 7 эпох
                model.train()  # Set model to training mode
            else:#иначе переходит в эволюцию и считаем качество валидации не переобучилась ли модель
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):#иттерируемся и получаем один батч и один тензор и переводим их на гпу
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()#обнуляем градиент чтобы они не накапливались

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):#если фаза равно трейн то мы активируем те градиенты которые не замороженны 
                    preds = model(inputs)#считаем предикшины
                    loss_value = loss(preds, labels)#посчитали лоссы 
                    preds_class = preds.argmax(dim=1)#что за класс предсказала наша сетка

                    # backward + optimize only if in training phase
                    if phase == 'train':#если фаза трейн 
                        loss_value.backward()#считаем производную
                        optimizer.step()#делаем градиентный шаг

                # statistics
                running_loss += loss_value.item()#считаются лоссы
                running_acc += (preds_class == labels.data).float().mean()#считаются точности

            epoch_loss = running_loss / len(dataloader) #делим на кол-во батчей
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

Строим модель RESENET18 (18 слоев)

In [ ]:
model = models.resnet18(pretrained=True)

#данный цикл обучает только последний слой нейросети где она отвечает за классификацию
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(model.fc.in_features, 2)#полносвязный линейный слой, поскольку RESENET ОН ДЛЯ 1000 КЛАССОВ А У НАС ИХ 2 В ПРИМЕРЕ
#model.fc.in_features - сколько фичей приходит на вход (нейронов) на выход нам нужно 2 нейрона
#переводи нашу архитектуру на ГПУ для быстроты вычислений
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

#планировщик для уменьшения градиентных шагов уменьшает каждые 7 эпох чтобы градиент не попопал локальный минимум
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train_model(model, loss, optimizer, scheduler, num_epochs=100); #модель тренировочная с 100 эпохами

In [ ]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
#класс который отдает путь к изображению
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
test_dataset

In [ ]:
model.eval()

test_predictions = []#классы предсказанных изображений 
test_img_paths = []#пути к изображению
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)#переводим на гпу
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)#считаем предикшины
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())#можем привритить к вероятности благодаря софтмакс
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)#соединяем предсказанные для тестовой выборки

In [ ]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

In [ ]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [ ]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

In [ ]:
submission_df.to_csv('submission.csv')

In [ ]:
!rm -rf train val test